# Лабораторная работа 1
# Тема 1. Эффективные инструменты в кольце классов вычетов

Шпак Андрей, 15.09.2022

# <font color='red'>Задание 1.</font>

Реализовать Быстрый алгоритм возведения в степень "слева направо". Оценить количество арифметических операций. Оценить временную сложность алгоритма. Является ли он полиномиальным? Сравнить фактическое время вычисления с функциями FastPower и pow.

## Реализация Быстрого алгоритма возведения в степень "слева направо".

### Материалы из лекции

#### Еще раз алгоритм быстрого возведения в степень "слева направо"


Пусть
$$
  d = d_02^k+d_12^{k-1}+\dots+d_{k-1}2+d_k,\qquad d_0=1.
$$  
Поэтому
$$
  a^d=a^{(\dots((2d_0+d_1)2+d_2)2+\dots+d_{k-1})2+d_k}.
$$
Вычисляя данную формулу "слева направо", получаем 

**Быстрый алгоритм возведения в степень "слева направо".**  
Вход: $a\in\mathbb{Z},\, d,m\in\mathbb{N}.$  
Выход: $b\in\mathbb{Z},$ $\qquad (0\leq b<m,\quad b\equiv a^d\mod m).$  
1$.\;\; b:=a.$  
2$.\;$ Для $\quad i:=1,2,\dots,k\quad$ вычисляем  
    $\qquad\quad b:=b^2a^{d_i} \mod m.$   
3$.\;$ Выдаем результат $b.$

#### Реализация быстрого алгоритма возведения в степень

In [72]:
def FastPower(a, d, m):
    b = 1
    while d>0:
        if d & 1:
            b = b*a % m
        a = a*a % m
        d = d >> 1
    return b

Если n - это степень двойки, $n = 2^k$, то для возведения в степень $n$ достаточно число возвести в квадрат $k$ раз, затратив при этом $k$ умножений вместо $2^k$

Например:
$n = 13 = 1101_2 = 1 * 2^3 + 1 * 2^2 + 0 * 2^1 + 1 * 2^0$

$a^n = a^{13} = a^{a^8 + a^4 + a^1}$

Количество умножений не проевосходит количество цифр в бинарном разложении числа $n$.

Пример ещё один:

$a^1 = a$

$a^2 = a^1 * a^1$

$a^4 = a^2 * a^2$

$a^8 = a^4 * a^4$

### Собственно, реализация Быстрого алгоритма возведения в степень "слева направо"

In [73]:
# Вопрос: Почему i начинается с 1?
# Ответ: Потому что изначально b уже равняется a, то есть этап a ** 1 выполнен. 

# Вариант со сдвигами:
def OldFastPower(a, d, m):
    b = a
    # стартовое значение b = a
    i = d
    # ввожу i, чтобы не изменять само d
    ln = len(bin(d)) - 3
    # чтобы не попадать в "начало" двоичного представления числа d и не трогать старший бит d_0 = 1
    j = 1
    while i > 1:
        b = (b ** 2) * (a ** (d >> (ln - j) & 1)) % m
        i = i >> 1
        j += 1
    return b

# Вариант попроще
def NewFastPower(a, d, m):
    b = a
    bin_lst = [int(i) for i in bin(d)[3:]]
    for elem in bin_lst:
        b = (b ** 2) * (a ** elem) % m
    return b

### Как я разбирался со сдвигами

In [74]:
# Тут я разбирался с тем, как работют побитовые сдвиги влево, вправо и побитовое и.
print(bin(2905))
len(bin(33))

# Получаю первый нужный бит.
2905 >> 11 & 1
# Сдвиг влево на 1 разряд увеличивает целое положительное число вдвое.
# Сдвиг вправо на 1  делит целое положительное число нацело на 2.
# Операция побитовое & с единичкой вернет последний бит.

# Длина, так как строковое представление содержит обозначение 0b.
num = 2905
ln = len(bin(num)) - 2

# Пробую побитово получать строку - битовое представление числа num
s = ''
for i in range(ln):
    s += str(num >> (ln - i - 1) & 1)
s

0b101101011001


'101101011001'

### Тесты

In [75]:
m = 131
a = 11122233344
d = 10**5
print(OldFastPower(a, d, m), NewFastPower(a, d, m), FastPower(a, d, m), a**d % m)
num, deg, mod = 245623, 2, 11
print(NewFastPower(num, deg, mod), OldFastPower(num, deg, mod), num ** deg % mod)
num, deg, mod = 61228935, 39025, 11111190
print(NewFastPower(num, deg, mod), OldFastPower(num, deg, mod), num ** deg % mod)
num, deg, mod = -534935, 3, 5555
print(NewFastPower(num, deg, mod), OldFastPower(num, deg, mod), num ** deg % mod)

99 99 99 99
5 5 5
9099915 9099915 9099915
4715 4715 4715


### Проблемы с единицей

In [76]:
print('Проблемы, если степень 1')
print(NewFastPower(2, 1, 2))
print(OldFastPower(2, 1, 2))
print(2 ** 1 % 2)

Проблемы, если степень 1
2
2
0


In [77]:
# если я учитываю, что a - целое и, что есть проблема с единицей, то
def NewFastPower(a, d, m):
    b = abs(a)
    if d == 1:
        return b % m
    bin_lst = [int(i) for i in bin(d)[3:]]
    for elem in bin_lst:
        b = (b ** 2) * (a ** elem) % m
    return b

In [78]:
print(NewFastPower(2, 1, 2))
print(OldFastPower(2, 1, 2))
print(2 ** 1 % 2)

0
2
0


## Оценка количества арифметических операций

На каждой итерации цикла Быстрого алгоритма возведения в степень "слева направо" выполняется 2 арифметических операции: умножение и вычисление остатка от деления, поэтому
$$
  f(a, d, m) = 2k.
$$
Возведение в степень арифметической операцией не считается (лекция).

## Оценка временной сложности работы алгоритма

$$
  \left<d\right>=k.
$$
$$
  T(N):=\max\limits_{\left<a,d,m\right>\leq N} f(a,d,m)\leq \max\limits_{\left<a,d,m\right>\leq N}2\left<d\right><СN.
$$

## Сравнение фактического времени выполнения с функциями FastPower, pow

In [79]:
import time
m = 131119345
a = 11121192354233344
d = 10**11

print('NewFastPower:')
st = time.time()
NewFastPower(a, d, m)
print(time.time() - st, '\n')

print('FastPower:')
st = time.time()
FastPower(a, d, m)
print(time.time() - st, '\n')

print('pow:')
st = time.time()
pow(a, d, m)
print(time.time() - st, '\n')

NewFastPower:
0.0 

FastPower:
0.000995635986328125 

pow:
0.0 



# <font color='red'>Задание 2.</font>

При помощи Расширенного алгоритма Евклида найти обратные к элементам $a,b\in\mathbb{Z}_m.$ Проверить результат. Сравнить с функцией `pow(a,-1,m)`. Оценить количество арифметических операций в Расширенном алгоритме Евклида. Оценить временную сложность алгоритма. Является ли он полиномиальным?

In [80]:
m = 42530430997171493050900585519445269701954006270353944787367883
a = -949014432282168334172
b = 2847043296846505002513

## Реализация поиска обратного элемента при помощи Расширенного алгоритма Евклида

### Материалы из лекции

#### Еще раз про вычисление обратного элемента
В кольце $\mathbb{Z}_m$ достаточно часто необходимо находить обратные относительно умножения элементы: $
  a*a^{-1}=1.
$

При помощи Расширенного алгоритма Евклида находим
$$
  u*a + v*m = НОД(a,m).
$$
Если $НОД(a,m)=1,$  то
$$
  u*a=1-v* m\equiv 1\mod m.
$$
Поэтому $a^{-1}=v.$ 

Можно доказать, что если $НОД(a,m)>1,$ то обратного элемента для $a$ в $\mathbb{Z}_m$ не существует.

#### Алгоритм Евклида

Вспоминаю на примере:
$$45 = 16 * 2 + 13$$
$$a = b * q1 + r1$$
$$16 = 13 * 1 + 3$$
$$b = r1 * q2 + r2$$
$$13 = 3 * 4 + 1$$
$$r1 = r2 * q3 + r3$$
$$3 = 1 * 3 + 0$$
$$r2 = r3 * q3 + r4$$
$$r4 = 0$$
$$r3$$

#### Расширенный алгоритм Евклида

На примере:
$$a = 45; \qquad b = 16$$
$$45 = 16*2 + 13$$
$$a = b*q1 + r1$$
$$a = 16; \qquad b = 13; \qquad M=\begin{pmatrix} 0 & 1 \\ 1 & -2\end{pmatrix}$$
$$16 = 13*1 + 3$$
$$b = r1*q2 + r2$$
$$a = 13; \qquad b = 3; \qquad M=\begin{pmatrix} 1 & -1 \\ -2 & 3\end{pmatrix}$$
$$13 = 3*4 + 1$$
$$r1 = r2*q3 + r3$$
$$a = 3; \qquad b = 1; \qquad M = \begin{pmatrix} -1 & 5 \\ 3 & -14\end{pmatrix}$$
$$3 = 1 * 3 + 0$$
$$r2 = r3*q4 + r4$$
$$r4 = 0; \qquad M = \begin{pmatrix} 5 & -16 \\ -14 & 45\end{pmatrix}$$
$$r3 = 1 = НОД(45, 16)$$
$$5*45 + (-14)*16 = 1$$

### Сама реализация поиска обратного элемента

In [81]:
def reverse_elem(a, b):
    if a == b - 1:
        return a % b
    a_copy = a
    b_copy = b
    m11, m12 = 1, 0
    m21, m22 = 0, 1
    while b:
        q = a // b
        a, b = b, a % b
        m11, m12 = m12, m11 - m12*q
        m21, m22 = m22, m21 - m22*q
    gcd = m11 * a_copy + m21 * b_copy
    if gcd == 1:
        return m11
    else:
        raise ValueError('base is not invertible for the given modulus')

## Проверка результата

In [82]:
print(reverse_elem(45, 16), pow(45, -1, 16))
print(reverse_elem(a, m))
print(pow(a, -1, m))

5 5
5760015108553103353761330938423529954070592368675905375141312
5760015108553103353761330938423529954070592368675905375141312


In [83]:
reverse_elem(b, m)

ValueError: base is not invertible for the given modulus

In [84]:
pow(b, -1, m)

ValueError: base is not invertible for the given modulus

## Оценка количества арифметических операций

В лекционном материале рассматривалась оценка количества остатков, вне зависимости от числа операций. Полагаю, что и в случае с использованием Расширенного алгоритма Евклида подход не изменяется, поэтому, сначала даю оценку количества итераций при $i = k$:
$$
G^{k-1}\leq r_{0}=b<2^{\left<b\right>}.
$$

$$
    (k-1)\log_2G<\left<b\right>,\qquad k<\frac{\left<b\right>}{\log_2G}+1.
$$
А затем утверждаю, что Расширенный алгоритм Евклида выполняет следующее количество арифметических операций
$$
  f(a, b) = k.
$$

## Оценка временной сложности работы алгоритма

Для временной сложности справедлива оценка
$$
T(N):=\max\limits_{\left<a,b\right>\leq N} f(a,b)< \frac{N}{\log_2G}+1
$$

## Расширенный алгоритм Евклида полиномиален

Видно, что, временная сложность расширенного алгоритма Евклида ограничена многочленом $\dfrac{N}{\log_2G}+1$, а длина чисел, участвующих в вычислениях алгоритма, ограничена $ k<\dfrac{\left<b\right>}{\log_2G}+1.$

# <font color='red'>Задание 3.</font>
Реализовать Алгоритм переполнения памяти умножением. Быстро ли работает данный алгоритм? Сколько арифметических операций выполняет данный алгоритм? Чему равна T(N)? Является ли он полиномиальным? 

## Реализация Алгоритма переполнения памяти умножением

### Материалы из лекции

#### Еще раз про Алгоритм переполнения памяти умножением

"Модернизируем" Наивный алгоритм возведения в степень. Теперь не будем вычислять остаток от деления.

**Алгоритм переполнения памяти умножением.**  
Вход: $n\in\mathbb{N}.$  
Выход: $2^{2^{[\log_2n]+1}}.$
1. Инициализируем переменную $a = 2.$
2. Для $i:=1,2,\dots,$ floor$(\log_2n)+1\;$ вычисляем $a:=a*a.$
3. Выдаем результат  $a.$


### Сама реализация алгоритма переполнения памяти умножением

In [85]:
import math

def multiplication_memory_overflow(n):
    a = 2
    edge = math.floor(math.log(n, 2)) + 1
    for i in range(edge):
        a *= a
    return a

### Проверка

In [86]:
num = 50
print(multiplication_memory_overflow(num))
print(pow(2, pow(2, math.floor(math.log(num, 2)) + 1)))

18446744073709551616
18446744073709551616


## Алгоритм переполнения памяти умножением экспоненциален (ИЗ ЛЕКЦИИ)

Подсчитаем количество арифметических операций, которые выполняются при работе Алгоритма переполнения памяти умножением
$$
  f(n)=floor(log_2𝑛)+1=\left<n\right>.
$$
Найдем временную сложность
$$
  T(N):=\max\limits_{\left<n\right>\leq N} f(n) = N.
$$
Функция $T(N)=N$ линейная, поэтому на первый взгляд кажется, что и Алгоритм переполнения памяти умножением также линеен. Но напомним точное определение полиномиального алгоритма:

Алгоритм $A$ с входными данными $n$ называется *полиномиальным*, если его временная сложность $T(N)$ ограничена многочленом $p(N),$ **а длина чисел, участвующих в вычислениях алгоритма, ограничена полиномом  $p(\left<n\right>).$** 

Результатом алгоритма является число $2^{2^{\left<n\right>}}.$ Найдем его длинну
$$
  \left<2^{2^{\left<n\right>}}\right>=\lceil\log_2(2^{2^{\left<n\right>}}+1)\rceil=2^{\left<n\right>}+1.
$$
Очевидно, что длина чисел, участвующих в вычислениях алгоритма,  выражается через экспоненту от длины входных данных и не может быть ограничена полиномом от $\left<n\right>.$ Поэтому алгоритм не является полиномиальным.

Такой алгоритм не является быстрым. https://habr.com/ru/company/vdsina/blog/544218/

# <font color='red'>Задание 4.</font>

Пусть временная сложность алгоритма $A_1$ имеет вид 
$$T_1(N)=2^N,$$ 
а алгоритма $A_2$ 
$$T_2(N)=N.$$
Пусть за один час работы компьютера $C_1$ можно выполнить алгоритм $A_1$ с длиной входных данных $N_1,$ а алгоритм $A_2$  с длиной входных данных $N_2.$  С какой длиной данных можно за 1 час выполнить алгоритмы $A_1$  и $A_2$ на компьютере $C_2$, производительность которого в два раза больше, чем $C_1$?  

В данной задаче считать, что все арифметические операции алгоритмов выполняются за одинаковое время, хоть в жизни это и не так, потому что с ростом входных данных приходиться проводить арифметические операции над числами большей длины.

## Решение

Количество операций не меняется, временная сложность тоже. Ничего лучше, чем указать, что за час на компьютере $C_2$ алгоритмы A1 и A2 могут работать с длинной данных $2N_1$ и $2^{N_2 + 1}$ соответственно. 

# <font color='red'>Задание 5.</font>

Выполнить компьютерное моделилорание кольца вычетов средствами ООП, перегрузив операторы `__add__, __sub__, __mul__, __truediv__, __pow__, __neg__.` Реализовать `__repr__, __eq__`. Проверить, обладает ли построенная модель свойствами:
1. $(\mathbb{Z}_m,+)$ - абелева группа:  
    1.1. Коммутативность сложения  
    $\qquad\qquad
      a + b = b + a.
    $  
    1.2. Ассоциативность сложения  
    $\qquad\qquad
      a + (b + с) = a + (b + c).
    $  
    1.3. Существование нейтрального элемента относительно сложения  
    $\qquad\qquad
      \exists0\in R \quad(a+0= 0+a = a).
    $  
    1.4. Существование противоположного элемента относительно сложения  
    $\qquad\qquad
      \forall a\in R\quad\exists b\in R \quad(a+b= b+a = 0).
    $
2. Ассоциативность умножения      
    $\qquad\qquad
      a * (b * с) = a * (b * c).
    $ 
3. Дистрибутивность    
$\qquad\qquad
  a*(b+c)=a*b+a*c\quad и\quad (b+c)*a=b*a+c*a.
$
4. Коммутативность умножения  
    $\qquad\qquad
      a * b = b * a.
    $  
5. Существование нейтрального элемента относительно умножения  
    $\qquad\qquad
      \exists1\in R \quad(1*a= a*1 = a).
    $  

6. Существование обратного элемента для ненулевых элементов  
$\qquad\qquad
      \forall a\in R\setminus \{0\} \quad\exists a^{-1}\in R \quad(a*a^{-1}= 1).
$

## Реализация класса

In [87]:
class Ring:
    def __init__(self, remainder, m):
        self.remainder = remainder
        self.mod = m

    def __repr__(self):
        return "(%s)" % (self.remainder)
    
    def __add__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder + other.remainder) % self.mod, self.mod)
        
    def __sub__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder - other.remainder) % self.mod, self.mod)
        
    def __mul__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder * other.remainder) % self.mod, self.mod)
        
    def __truediv__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder / other.remainder) % self.mod, self.mod)
        
    def __pow__(self, num):
        return Ring(NewFastPower(self.remainder, num, self.mod), self.mod)
        
    def __neg__(self):
        return Ring(reverse_elem(self.remainder, self.mod), self.mod)
        
    def __eq__(self, other):
        if isinstance(other, Ring):
            if self.remainder == other.remainder:
                return True
            else:
                return False

## Пробую работать с экземплярами классов

In [103]:
a = Ring(0, 3)
b = Ring(1, 3)
c = Ring(2, 3)

In [89]:
c - c * a

(2)

In [90]:
c == a

False

In [91]:
-c

(2)

In [92]:
pow(2, -1, 3)

2

In [107]:
c ** -2

(1)

## А может ли построенная модель оказаться полем?

In [101]:
import random
m = random.randint(1, 100)
Z_m = [Ring(i, m) for i in range(m)]
Z_m

[(0),
 (1),
 (2),
 (3),
 (4),
 (5),
 (6),
 (7),
 (8),
 (9),
 (10),
 (11),
 (12),
 (13),
 (14),
 (15),
 (16),
 (17),
 (18),
 (19),
 (20),
 (21),
 (22),
 (23),
 (24),
 (25),
 (26),
 (27),
 (28),
 (29),
 (30),
 (31),
 (32),
 (33),
 (34),
 (35),
 (36),
 (37),
 (38),
 (39),
 (40),
 (41),
 (42),
 (43),
 (44),
 (45),
 (46),
 (47),
 (48),
 (49),
 (50),
 (51),
 (52),
 (53),
 (54),
 (55),
 (56),
 (57),
 (58),
 (59),
 (60),
 (61),
 (62),
 (63),
 (64),
 (65),
 (66),
 (67),
 (68),
 (69),
 (70),
 (71),
 (72),
 (73),
 (74),
 (75),
 (76),
 (77),
 (78),
 (79),
 (80),
 (81),
 (82),
 (83),
 (84),
 (85),
 (86),
 (87),
 (88),
 (89),
 (90),
 (91)]

In [98]:
a, b = random.randint(0, m - 1), random.randint(0, m - 1)
d = random.randint(0, m - 1)
print('Коммутативность сложения:')
print(Z_m[a] + Z_m[b] == Z_m[b] + Z_m[a])
print('Ассоциативность сложения:')
print(Z_m[a] + (Z_m[b] + Z_m[d]) == Z_m[a] + (Z_m[b] + Z_m[d]))
print('Существование нейтрального элемента относительно сложения:')
print(Z_m[a] + Z_m[0] == Z_m[0] + Z_m[a] == Z_m[a])
print('Существование противоположного элемента относительно сложения:')
print(Z_m[a] - Z_m[a] == Z_m[a] - Z_m[a] == Z_m[0])

print('Ассоциативность умножения:')
print(Z_m[a] * (Z_m[b] * Z_m[d]) == Z_m[a] * (Z_m[b] * Z_m[d]))
print('Дистрибутивность умножения:')
print(Z_m[a] * (Z_m[b] + Z_m[d]) == Z_m[a] * Z_m[b] + Z_m[a] * Z_m[d])
print((Z_m[b] + Z_m[d]) * Z_m[a] == Z_m[b] * Z_m[a] + Z_m[d] * Z_m[a])
print('Коммутативность сложения:')
print(Z_m[a] * Z_m[b] == Z_m[b] * Z_m[a])
print('Существование нейтрального элемента относительно умножения:')
print(Z_m[a] * Z_m[1] == Z_m[1] * Z_m[a] == Z_m[a])
print('Существование обратного элемента для ненулевых элементов:')
try:
    print(Z_m[a] * (-Z_m[a]) == (-Z_m[a]) * Z_m[a] == Z_m[1])
except(ValueError):
    print(False)

Коммутативность сложения:
True
Ассоциативность сложения:
True
Существование нейтрального элемента относительно сложения:
True
Существование противоположного элемента относительно сложения:
True
Ассоциативность умножения:
True
Дистрибутивность умножения:
True
True
Коммутативность сложения:
True
Существование нейтрального элемента относительно умножения:
True
Существование обратного элемента для ненулевых элементов:
False
